# **Evaluacion de Data set**
Tecnicatura en inteligencia artificial

Universidad Nacional de Hurlingham

Docente: Andrea Rey

Alumno : Nicolas Seivane

Librerias que voy a utilizar


In [26]:
import pandas as pd
import sklearn as sk
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    matthews_corrcoef,
    balanced_accuracy_score,
    log_loss,
    cohen_kappa_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import StratifiedKFold# .head()
# .info()
# .isnull().any()
# df.column.value_counts()
# .max()
# .min()
# .describe()

In [27]:
datos = pd.read_csv("C:\\Users\\User\\Documents\\Proyecto integrador\\heart\\heart.csv")

datos.drop_duplicates(inplace=True)
datos.dropna(inplace=True)
#datos.drop(columns=["CLASS"], inplace=True)
#datos['NSP'] = datos['NSP'].replace({1: 'Normal', 2: 'Sospechoso', 3: 'Patologico'})
datos.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [28]:
datos.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [29]:
datos.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [30]:
n_filas = len(datos)

for column in datos.columns:
    unicos = datos[column].unique()
    n_unicos = len(unicos)
    
    # Si es categórica (tipo object/category) o es numérica con pocos únicos
    if (datos[column].dtype == 'object') or (len(unicos) <= 10):
        print(f"\nColumna '{column}' (candidata categórica): {n_unicos} valores únicos")
        
        
        for unico in unicos:
                count = len(datos[datos[column] == unico])
                normalizado = count / n_filas
                print(f"   Hay {normalizado:.2%} codificado en {unico}")
    else:
        print(f"\nColumna '{column}' tiene media: {datos[column].mean():.2f}, valor maximo: {datos[column].max():.2f} y valor minimo: {datos[column].min():.2f}")


Columna 'Age' tiene media: 53.51, valor maximo: 77.00 y valor minimo: 28.00

Columna 'Sex' (candidata categórica): 2 valores únicos
   Hay 78.98% codificado en M
   Hay 21.02% codificado en F

Columna 'ChestPainType' (candidata categórica): 4 valores únicos
   Hay 18.85% codificado en ATA
   Hay 22.11% codificado en NAP
   Hay 54.03% codificado en ASY
   Hay 5.01% codificado en TA

Columna 'RestingBP' tiene media: 132.40, valor maximo: 200.00 y valor minimo: 0.00

Columna 'Cholesterol' tiene media: 198.80, valor maximo: 603.00 y valor minimo: 0.00

Columna 'FastingBS' (candidata categórica): 2 valores únicos
   Hay 76.69% codificado en 0
   Hay 23.31% codificado en 1

Columna 'RestingECG' (candidata categórica): 3 valores únicos
   Hay 60.13% codificado en Normal
   Hay 19.39% codificado en ST
   Hay 20.48% codificado en LVH

Columna 'MaxHR' tiene media: 136.81, valor maximo: 202.00 y valor minimo: 60.00

Columna 'ExerciseAngina' (candidata categórica): 2 valores únicos
   Hay 59.59% 

In [31]:
for column in datos.columns:
    if len(datos[column].unique()) <= 10 and datos[column].dtype == 'object':
        print(column, datos[column].unique())



Sex ['M' 'F']
ChestPainType ['ATA' 'NAP' 'ASY' 'TA']
RestingECG ['Normal' 'ST' 'LVH']
ExerciseAngina ['N' 'Y']
ST_Slope ['Up' 'Flat' 'Down']


In [32]:
for column in datos.columns:
    if len(datos[column].unique()) <= 10 and datos[column].dtype == 'object':
        unicos = datos[column].unique()
        for i in range(len(unicos)):
            datos[column] = datos[column].replace({unicos[i]: i})
        print(column, datos[column].unique())

Sex [0 1]
ChestPainType [0 1 2 3]
RestingECG [0 1 2]
ExerciseAngina [0 1]
ST_Slope [0 1 2]


C:\Users\User\AppData\Local\Temp\ipykernel_18364\495973239.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos[column] = datos[column].replace({unicos[i]: i})


In [33]:
for column in datos.columns:
    if len(datos[column].unique()) > 4:
        datos[column] = (datos[column] - datos[column].mean()) / datos[column].std()

In [34]:
atributos = datos.columns[:-1]
concepto = datos.columns[-1]

datos.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,-1.432359,0,0,0.410685,0.824621,0,0,1.382175,0,-0.831979,0,0
1,-0.478223,1,1,1.490940,-0.171867,0,0,0.753746,0,0.105606,1,1
2,-1.750404,0,0,-0.129442,0.769768,0,1,-1.524307,0,-0.831979,0,0
3,-0.584238,1,2,0.302660,0.138964,0,0,-1.131539,1,0.574398,1,1
4,0.051853,0,1,0.950812,-0.034736,0,0,-0.581664,0,-0.831979,0,0


In [35]:
def validacion_cruzada(conjunto, atributos, concepto, k=5, random=False, agregar_unos=False, undersample=False, oversample=False):
  
  if undersample:
    cantidad_menor_concepto = conjunto[concepto].value_counts().min()
    conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_menor_concepto, random_state=42) for valor in conjunto[concepto].unique()])

  if oversample:
    cantidad_mayor_concepto = conjunto[concepto].value_counts().max()
    conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_mayor_concepto, replace=True, random_state=42) for valor in conjunto[concepto].unique()])

  if random: conjunto = conjunto.sample(frac=1, random_state=11).reset_index(drop=True)

  x = conjunto[atributos]
  if agregar_unos: x.insert(0, "Unos", 1)

  y = conjunto[concepto]

  resultados = []
  skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

  for train_index, test_index in skf.split(x, y):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        resultados.append((x_train, y_train, x_test, y_test))

  return resultados

from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier(
    n_estimators=100,      # número de árboles en el bosque
    criterion="gini",      # función de calidad de la división ("gini" o "entropy")
    max_depth=None,        # profundidad máxima de cada árbol
    min_samples_split=2,   # min muestras para dividir un nodo
    min_samples_leaf=1,    # min muestras en una hoja
    max_features="sqrt",   # nº de features a considerar en cada split ("sqrt", "log2", None)
    bootstrap=True,        # si usar bootstrap samples para entrenar cada árbol
    random_state=42,       # semilla
    n_jobs=-1              # usa todos los cores
)

from sklearn.naive_bayes import GaussianNB

GaussianNB(
    var_smoothing=1e-9     # suavizado para evitar divisiones por cero
)


from sklearn.svm import SVC

SVC(
    C=1.0,                 # penalización del error (mayor C = menos margen, más sobreajuste)
    kernel="rbf",          # kernel: "linear", "poly", "rbf", "sigmoid"
    degree=3,              # grado del polinomio (si kernel="poly")
    gamma="scale",         # controla la influencia de un solo ejemplo ("scale", "auto" o un valor numérico)
    probability=True,     # si calcular probabilidades (más lento)
    random_state=42
)

from sklearn.linear_model import LogisticRegression

LogisticRegression(
    penalty="l2",          # regularización: "l1", "l2", "elasticnet", None
    C=1.0,                 # inverso de la fuerza de regularización (menor C = más regularización)
    solver="lbfgs",        # optimizador ("lbfgs", "saga", "liblinear", "newton-cg")
    max_iter=1000,         # iteraciones máximas (importante en datasets grandes)
    multi_class="multinomial",    # "ovr" (uno-vs-rest) o "multinomial"
    random_state=42
)

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    matthews_corrcoef,
    balanced_accuracy_score,
    log_loss,
    cohen_kappa_score
)

    y_true_total.extend(y_test)
    y_pred_total.extend(y_pred)

print("Matriz de confusión total:\n", confusion_matrix(y_true_total, y_pred_total))

LogisticRegression(
    penalty="l2",          # regularización: "l1", "l2", "elasticnet", None
    C=1.0,                 # inverso de la fuerza de regularización (menor C = más regularización)
    solver="lbfgs",        # optimizador ("lbfgs", "saga", "liblinear", "newton-cg")
    max_iter=1000,         # iteraciones máximas (importante en datasets grandes)
    multi_class="multinomial",    # "ovr" (uno-vs-rest) o "multinomial"
    random_state=42
)

# *Regresion Logistica*

In [36]:
grid = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l1","l2","elasticnet", None],  # 'l1' si usas solver='liblinear' o 'saga'
    "solver": ["lbfgs","saga", "newton-cg"],# según el penalty que uses, liblinear no soporta None, newton-cg no soporta l1 ni elasticnet
    "multi_class": ["ovr", "multinomial"] # si es multiclass
}
 
## cohen_kappa_score,     accuracy_score,    precision_score,     recall_score,    f1_score,    roc_auc_score,


#with open("regresion_logistica_resultados.txt", "w") as f:
with open("regresion_logistica_resultados_corazon.txt", "w") as f:
    f.write("Resultados de Regresion Logistica con validacion cruzada (5 folds):\n")
    
    for C in grid["C"]:

        for penalty in grid["penalty"]:
            for multi_class in grid["multi_class"]:
                for solver in grid["solver"]:
                    # chequeo de compatibilidad
                    if penalty == "l1" and solver not in ["liblinear", "saga"]:
                        continue
                    if penalty == "elasticnet" and solver != "saga":
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    scoring_funcs = {
                                "cohen_kappa": 0,
                                "accuracy": 0,
                                "precision": 0,
                                "recall": 0,
                                "f1": 0,
                                "roc_auc": 0
                            }
                    try:
                        inicio = time.time()
                        modelo = LogisticRegression(
                            C=C,
                            penalty=penalty,
                            solver=solver,
                            l1_ratio=0.5 if penalty=="elasticnet" else None,
                            multi_class=multi_class,
                            max_iter=1000,
                            random_state=42
                        )

                        folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)
                        
                        for x_train, y_train, x_test, y_test in folds:
                                modelo.fit(x_train, y_train)
                                pred = modelo.predict(x_test)
                                scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                probs = modelo.predict_proba(x_test)
                                if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                
                        fin = time.time()
                        tiempo_total = fin - inicio
                        for key in scoring_funcs:
                            scoring_funcs[key] /= len(folds) 
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                    except Exception as e:
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Regresion Logistica con validacion cruzada (5 folds) undersample:\n")
    
    for C in grid["C"]:

        for penalty in grid["penalty"]:
            for solver in grid["solver"]:
                for multi_class in grid["multi_class"]:
                    # chequeo de compatibilidad
                    if penalty == "l1" and solver not in ["liblinear", "saga"]:
                        continue
                    if penalty == "elasticnet" and solver != "saga":
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    scoring_funcs = {
                                "cohen_kappa": 0,
                                "accuracy": 0,
                                "precision": 0,
                                "recall": 0,
                                "f1": 0,
                                "roc_auc": 0
                            }
                    try:
                        tiempo_inicio = time.time()
                        modelo = LogisticRegression(
                            C=C,
                            penalty=penalty,
                            solver=solver,
                            l1_ratio=0.5 if penalty=="elasticnet" else None,
                            multi_class=multi_class,
                            max_iter=1000,
                            random_state=42
                        )

                        folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                        for x_train, y_train, x_test, y_test in folds:
                                modelo.fit(x_train, y_train)
                                pred = modelo.predict(x_test)
                                scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                probs = modelo.predict_proba(x_test)
                                if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")


                        fin = time.time()
                        tiempo_total = fin - tiempo_inicio
                        for key in scoring_funcs:
                            scoring_funcs[key] /= len(folds)  # Promediar las métricas

                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                    except Exception as e:
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> ERROR: {str(e)}\n")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\User\AppData\

RandomForestClassifier(
    n_estimators=100,      # número de árboles en el bosque
    criterion="gini",      # función de calidad de la división ("gini" o "entropy")
    max_depth=None,        # profundidad máxima de cada árbol
    min_samples_split=2,   # min muestras para dividir un nodo
    min_samples_leaf=1,    # min muestras en una hoja
    max_features="sqrt",   # nº de features a considerar en cada split ("sqrt", "log2", None)
    bootstrap=True,        # si usar bootstrap samples para entrenar cada árbol
    random_state=42,       # semilla
    n_jobs=-1              # usa todos los cores
)


# *Random Forest*

In [37]:
grid = {
    "n_estimators": [50, 100, 200, 500, 1000],
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 3, 5, 7, 9],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [None, "sqrt", "log2"]
}



#with open("Random_forest.txt", "w") as f:
with open("Random_forest_corazon.txt", "w") as f:
    f.write("Resultados de Random Forest con validacion cruzada (5 folds):\n")

    for criterion in grid["criterion"]:
        for max_depth in grid["max_depth"]:
            for min_samples_split in grid["min_samples_split"]:
                for min_samples_leaf in grid["min_samples_leaf"]:
                    for max_features in grid["max_features"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = RandomForestClassifier(
                                criterion=criterion,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                max_features=max_features,
                                random_state=42
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} "
                                    f"--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")
                        except Exception as e:
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Random Forest con validacion cruzada (5 folds) undersample:\n")
    
    for criterion in grid["criterion"]:
        for max_depth in grid["max_depth"]:
            for min_samples_split in grid["min_samples_split"]:
                for min_samples_leaf in grid["min_samples_leaf"]:
                    for max_features in grid["max_features"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = RandomForestClassifier(
                                criterion=criterion,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                max_features=max_features,
                                random_state=42
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase → usar esquema OVR
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} "
                                    f"--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")
                        except Exception as e:
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} --> ERROR: {str(e)}\n")

GaussianNB(
    var_smoothing=1e-9     # suavizado para evitar divisiones por cero
)


# *Naive Bayes*

In [38]:
grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]  
}

#with open("Naiva_Bayes.txt", "w") as f:
with open("Naiva_Bayes_corazon.txt", "w") as f:
    f.write("Resultados de Naive Bayes con validacion cruzada (5 folds):\n")

    for var_smoothing in grid["var_smoothing"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = GaussianNB(var_smoothing=var_smoothing)

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\n var_smoothing={var_smoothing}--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\n var_smoothing={var_smoothing}--> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Naive Bayes con validacion cruzada (5 folds) undersample:\n")

    for var_smoothing in grid["var_smoothing"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = GaussianNB(var_smoothing=var_smoothing)
                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)
                                    if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\n var_smoothing={var_smoothing}--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\n var_smoothing={var_smoothing}--> ERROR: {str(e)}\n")

SVC(
    C=1.0,                 # penalización del error (mayor C = menos margen, más sobreajuste)
    kernel="rbf",          # kernel: "linear", "poly", "rbf", "sigmoid"
    degree=3,              # grado del polinomio (si kernel="poly")
    gamma="scale",         # controla la influencia de un solo ejemplo ("scale", "auto" o un valor numérico)
    probability=True,     # si calcular probabilidades (más lento)
    random_state=42
)


# *SVM*

In [39]:
grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 15, 20, 25],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "gamma": ["scale", "auto"],
    "degree": [2,3,4,5,6,7,8,9,10]
}

#with open("SVM.txt", "w") as f:
with open("SVM_corazon.txt", "w") as f:
    f.write("Resultados de SVM con validacion cruzada (5 folds):\n")

    for C in grid["C"]:
        for kernel in grid["kernel"]:
            for gamma in grid["gamma"]:
                degrees_to_use = grid["degree"] if kernel == "poly" else [3]
                for degree in degrees_to_use:
                        scoring_funcs = {
                                "cohen_kappa": 0,
                                 "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = SVC(
                                  C=C,
                                    kernel=kernel,
                                    gamma=gamma,
                                    degree=degree,
                                    max_iter=1000,
                                    random_state=42,
                                    probability=True
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)

                                    if len(set(y_test)) == 2:
                                        # binaria
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\nC={C}, kernel={kernel}, gamma={gamma}, degree={degree} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\nC={C}, kernel={kernel}, gamma={gamma}, degree={degree} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de SVM con validacion cruzada (5 folds) undersample:\n")

    for C in grid["C"]:
        for kernel in grid["kernel"]:
            for gamma in grid["gamma"]:
                if kernel != "poly" and "degree" in grid:
                    continue
                for degree in grid["degree"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    "roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = SVC(
                                  C=C,
                                    kernel=kernel,
                                    gamma=gamma,
                                    degree=degree if kernel=="poly" else None,
                                    max_iter=1000,
                                    random_state=42,
                                    probability=True
                            )
                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    probs = modelo.predict_proba(x_test)

                                    if len(set(y_test)) == 2:
                                        # binaria
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    else:
                                        # multiclase
                                        scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\nC={C}, C={C}, gamma={gamma}, degree={degree} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\nC={C}, C={C}, gamma={gamma}, degree={degree} --> ERROR: {str(e)}\n")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag